In [1]:
import numpy as np
import pandas as pd

Max reward problem

In [2]:
pi = np.array([1, 2])

# A[i,j] = the reward you get when transist from i to j
A = np.array([[3, 2],
              [1, 4]])

B = np.array([[4, 7, 5, 4],
              [6, 1, 5, 3]])

In [3]:
def get_reward(seq, pi, A, B):
    reward = pi[seq[0]] + B[seq[0], 0]
    for i in [1, 2, 3]:
        reward += A[seq[i-1], seq[i]] + B[seq[i], i]

    return reward

In [4]:
seq = [1, 0, 1, 1]
get_reward(seq, pi, A, B)

30

Naive implementation

In [5]:
for x in [0, 1]:
    for y in [0, 1]:
        for z in [0, 1]:
            for w in [0, 1]:
                seq = [x, y, z, w]
                print(x, y, z, w, get_reward(seq, pi, A, B))

0 0 0 0 30
0 0 0 1 28
0 0 1 0 27
0 0 1 1 29
0 1 0 0 21
0 1 0 1 19
0 1 1 0 22
0 1 1 1 24
1 0 0 0 31
1 0 0 1 29
1 0 1 0 28
1 0 1 1 30
1 1 0 0 26
1 1 0 1 24
1 1 1 0 27
1 1 1 1 29


Dynamic programming

In [6]:
alpha = np.zeros((2, 4)) # alpha[i, j] = the maximum reward you can get from START to node[i,j]
backp = np.zeros((2, 4), dtype=int) # to achieve the maximum reward, you should come from node[backp[i,j], j-1]

In [7]:
alpha[:, 0] = pi + B[:, 0]
for i in [1, 2, 3]: # the current col
    for tag_i in range(2): # tag i
        rewards = alpha[:, i-1] + A[:, tag_i] + B[tag_i, i]

        alpha[tag_i, i] = np.max(rewards)
        backp[tag_i, i] = np.argmax(rewards)

In [8]:
max_reward = np.max(alpha[:, 3])
print(max_reward)

31.0


In [9]:
last_tag = np.argmax(alpha[:, 3])
seq = [last_tag]
for i in [3, 2, 1]:
    last_tag = backp[last_tag, i]
    seq.append(last_tag)
print(seq[::-1])

[1, 0, 0, 0]


Put above cells together in a function

In [10]:
def decode(pi, A, B):
    num_tags, num_words = B.shape
    
    alpha = np.zeros((num_tags, num_words))
    backp = np.zeros((num_tags, num_words), dtype=int)
    
    alpha[:, 0] = pi + B[:, 0]
    for i in range(1, num_words): # the current col
        for tag_i in range(num_tags): # tag i
            rewards = alpha[:, i-1] + A[:, tag_i] + B[tag_i, i]

            alpha[tag_i, i] = np.max(rewards)
            backp[tag_i, i] = np.argmax(rewards)
            
    last_tag = np.argmax(alpha[:, 3])
    seq = [last_tag]
    for i in range(num_words-1, 0, -1): # num_words-1, ..., 2, 1
        last_tag = backp[last_tag, i]
        seq.append(last_tag)
    return seq[::-1]

In [11]:
decode(pi, A, B)

[1, 0, 0, 0]

Real-world example: estimate pi, A, B from the treebank corpus

In [12]:
from nltk.corpus import treebank

In [13]:
corpus = treebank.tagged_sents()
print(corpus)

[[('Pierre', 'NNP'), ('Vinken', 'NNP'), (',', ','), ('61', 'CD'), ('years', 'NNS'), ('old', 'JJ'), (',', ','), ('will', 'MD'), ('join', 'VB'), ('the', 'DT'), ('board', 'NN'), ('as', 'IN'), ('a', 'DT'), ('nonexecutive', 'JJ'), ('director', 'NN'), ('Nov.', 'NNP'), ('29', 'CD'), ('.', '.')], [('Mr.', 'NNP'), ('Vinken', 'NNP'), ('is', 'VBZ'), ('chairman', 'NN'), ('of', 'IN'), ('Elsevier', 'NNP'), ('N.V.', 'NNP'), (',', ','), ('the', 'DT'), ('Dutch', 'NNP'), ('publishing', 'VBG'), ('group', 'NN'), ('.', '.')], ...]


In [14]:
from collections import defaultdict
tag_word_counts = defaultdict(lambda: defaultdict(lambda: 0))
tag_tag_counts  = defaultdict(lambda: 0)

for sent in corpus:
    last_tag = 'START'
    for word, tag in sent:
        tag_word_counts[tag][word.lower()] += 1
        tag_tag_counts[last_tag, tag] += 1
        last_tag = tag

calculate the transition matrix A

In [15]:
tags = []
word_dicts = []
for tag, word_dict in tag_word_counts.items():
    tags.append(tag)
    word_dicts.append(word_dict)

In [16]:
from sklearn.feature_extraction import DictVectorizer

vectorizer = DictVectorizer()
count_matrix = vectorizer.fit_transform(word_dicts)

words = vectorizer.feature_names_
count_matrix = count_matrix.toarray()

In [17]:
df_emission = pd.DataFrame(count_matrix, index=tags, columns=words)

df_emission += 1e-6 # smooth, add a small value to the whole matrix
df_emission = df_emission.div(df_emission.sum(axis=1), axis=0) # normalize, so that every row sums up to 1

In [18]:
df_emission

,!,#,$,%,&,','','30s,'40s,'50s,...,zealand,zenith,zero,zicklin,zinc,zip,zone,zoomed,zuckerman,zurich
CD,2.820070e-10,2.820070e-10,2.820070e-10,2.820070e-10,2.820070e-10,2.820070e-10,2.820070e-10,2.820073e-04,2.820073e-04,2.820073e-04,...,2.820070e-10,2.820070e-10,2.820073e-04,2.820070e-10,2.820070e-10,2.820070e-10,2.820070e-10,2.820070e-10,2.820070e-10,2.820070e-10
NNP,1.062698e-10,1.062698e-10,1.062698e-10,1.062698e-10,1.062698e-10,1.062698e-10,1.062698e-10,1.062698e-10,1.062698e-10,1.062698e-10,...,1.062699e-04,2.125397e-04,1.062698e-10,1.062699e-04,1.062698e-10,1.062698e-10,1.062698e-10,1.062698e-10,1.062699e-04,1.062699e-04
VBZ,4.705857e-10,4.705857e-10,4.705857e-10,4.705857e-10,4.705857e-10,4.705857e-10,4.705857e-10,4.705857e-10,4.705857e-10,4.705857e-10,...,4.705857e-10,4.705857e-10,4.705857e-10,4.705857e-10,4.705857e-10,4.705857e-10,4.705857e-10,4.705857e-10,4.705857e-10,4.705857e-10
WP,4.149182e-09,4.149182e-09,4.149182e-09,4.149182e-09,4.149182e-09,4.149182e-09,4.149182e-09,4.149182e-09,4.149182e-09,4.149182e-09,...,4.149182e-09,4.149182e-09,4.149182e-09,4.149182e-09,4.149182e-09,4.149182e-09,4.149182e-09,4.149182e-09,4.149182e-09,4.149182e-09
PDT,3.702142e-08,3.702142e-08,3.702142e-08,3.702142e-08,3.702142e-08,3.702142e-08,3.702142e-08,3.702142e-08,3.702142e-08,3.702142e-08,...,3.702142e-08,3.702142e-08,3.702142e-08,3.702142e-08,3.702142e-08,3.702142e-08,3.702142e-08,3.702142e-08,3.702142e-08,3.702142e-08
TO,4.589237e-10,4.589237e-10,4.589237e-10,4.589237e-10,4.589237e-10,4.589237e-10,4.589237e-10,4.589237e-10,4.589237e-10,4.589237e-10,...,4.589237e-10,4.589237e-10,4.589237e-10,4.589237e-10,4.589237e-10,4.589237e-10,4.589237e-10,4.589237e-10,4.589237e-10,4.589237e-10
RB,3.543572e-10,3.543572e-10,3.543572e-10,3.543572e-10,3.543572e-10,3.543572e-10,3.543572e-10,3.543572e-10,3.543572e-10,3.543572e-10,...,3.543572e-10,3.543572e-10,3.543572e-10,3.543572e-10,3.543572e-10,3.543572e-10,3.543572e-10,3.543572e-10,3.543572e-10,3.543572e-10
-NONE-,1.516988e-10,1.516988e-10,1.516988e-10,1.516988e-10,1.516988e-10,1.516988e-10,1.516988e-10,1.516988e-10,1.516988e-10,1.516988e-10,...,1.516988e-10,1.516988e-10,1.516988e-10,1.516988e-10,1.516988e-10,1.516988e-10,1.516988e-10,1.516988e-10,1.516988e-10,1.516988e-10
$,1.381194e-09,1.381194e-09,9.916971e-01,1.381194e-09,1.381194e-09,1.381194e-09,1.381194e-09,1.381194e-09,1.381194e-09,1.381194e-09,...,1.381194e-09,1.381194e-09,1.381194e-09,1.381194e-09,1.381194e-09,1.381194e-09,1.381194e-09,1.381194e-09,1.381194e-09,1.381194e-09
DT,1.224738e-10,1.224738e-10,1.224738e-10,1.224738e-10,1.224738e-10,1.224738e-10,1.224738e-10,1.224738e-10,1.224738e-10,1.224738e-10,...,1.224738e-10,1.224738e-10,1.224738e-10,1.224738e-10,1.224738e-10,1.224738e-10,1.224738e-10,1.224738e-10,1.224738e-10,1.224738e-10


In [19]:
sent = 'eye drops off shelf'.split() # an example sentence

In [20]:
df_emission[sent] # extract related columns from the full maxtrix, this is the matrix we will use

,eye,drops,off,shelf
CD,2.820070e-10,2.820070e-10,2.820070e-10,2.820070e-10
NNP,1.062698e-10,1.062698e-10,1.062698e-10,1.062698e-10
VBZ,4.705857e-10,4.705857e-10,4.705857e-10,4.705857e-10
WP,4.149182e-09,4.149182e-09,4.149182e-09,4.149182e-09
PDT,3.702142e-08,3.702142e-08,3.702142e-08,3.702142e-08
TO,4.589237e-10,4.589237e-10,4.589237e-10,4.589237e-10
RB,3.543572e-10,3.543572e-10,1.417429e-03,3.543572e-10
-NONE-,1.516988e-10,1.516988e-10,1.516988e-10,1.516988e-10
$,1.381194e-09,1.381194e-09,1.381194e-09,1.381194e-09
DT,1.224738e-10,1.224738e-10,1.224738e-10,1.224738e-10


calculate the transition matrix and the pi vector

In [21]:
df_trans = pd.DataFrame(columns=tags, index=tags)
df_pi    = pd.Series(index=tags)

for t1, t2 in tag_tag_counts:
    if t1 == 'START':
        df_pi.loc[t2] = tag_tag_counts[t1, t2]
    else:
        df_trans.loc[t1, t2] = tag_tag_counts[t1, t2]
        
df_pi = df_pi.fillna(0) + 1e-6 # smooth
df_pi = df_pi.div(df_pi.sum()) # normalize

df_trans = df_trans.fillna(0) + 1e-6 # smooth
df_trans = df_trans.div(df_trans.sum(axis=1), axis=0) # normalize

In [22]:
df_pi

CD        8.431273e-03
NNP       1.977517e-01
VBZ       2.299438e-03
WP        3.576904e-03
PDT       7.664796e-04
TO        1.277466e-03
RB        4.471129e-02
-NONE-    2.095043e-02
$         1.277466e-03
DT        2.312213e-01
IN        1.290240e-01
,         2.554931e-10
''        2.554934e-04
JJS       1.532959e-03
VBP       2.554931e-10
RBS       5.109865e-04
RP        2.554931e-10
VB        7.664796e-04
VBN       1.788452e-03
NNS       4.675524e-02
WRB       6.387328e-03
NN        4.445580e-02
WP$       2.554931e-10
JJ        3.653551e-02
:         2.810424e-03
MD        2.554934e-04
``        7.562596e-02
RBR       7.664796e-04
LS        1.788452e-03
CC        5.135411e-02
#         2.554931e-10
-RRB-     2.554931e-10
-LRB-     1.788452e-03
PRP       6.259581e-02
VBG       4.343383e-03
EX        4.343383e-03
FW        2.554931e-10
WDT       5.109865e-04
POS       2.554931e-10
VBD       2.554934e-04
PRP$      7.409300e-03
SYM       2.554931e-10
UH        2.554934e-04
.         2

In [23]:
df_trans

,CD,NNP,VBZ,WP,PDT,TO,RB,-NONE-,$,DT,...,FW,WDT,POS,VBD,PRP$,SYM,UH,.,JJR,NNPS
CD,1.851016e-01,1.185102e-02,2.257337e-03,2.821673e-04,2.821670e-10,2.624153e-02,1.975170e-03,2.104966e-01,2.821670e-10,1.410835e-03,...,2.821670e-10,1.693003e-03,8.465014e-04,5.361174e-03,2.821673e-04,2.821670e-10,2.821670e-10,4.937923e-02,8.465014e-04,2.821670e-10
NNP,2.020632e-02,3.825375e-01,3.669042e-02,6.380943e-04,1.063490e-10,4.147613e-03,7.657131e-03,5.636499e-03,2.126982e-04,2.233330e-03,...,1.063490e-10,5.317453e-04,5.019675e-02,6.476656e-02,1.063490e-10,1.063491e-04,1.063490e-10,5.051579e-02,1.063491e-04,1.722854e-02
VBZ,1.694118e-02,1.976471e-02,4.705887e-04,1.411765e-03,4.705887e-04,6.588236e-03,1.312941e-01,1.896471e-01,6.117647e-03,1.397647e-01,...,4.705882e-10,4.705882e-10,4.705882e-10,1.411765e-03,8.470589e-03,4.705882e-10,4.705882e-10,2.823530e-03,7.058824e-03,4.705882e-10
WP,4.149381e-03,2.074689e-02,1.659751e-02,4.149377e-09,4.149381e-03,4.149377e-09,1.659751e-02,7.800828e-01,4.149377e-09,4.149377e-02,...,4.149377e-09,4.149377e-09,4.149377e-09,4.149377e-09,1.244813e-02,4.149377e-09,4.149377e-09,4.149377e-09,4.149377e-09,4.149377e-09
PDT,3.703697e-08,3.703697e-08,3.703697e-08,3.703697e-08,3.703697e-08,3.703697e-08,3.703697e-08,3.703697e-08,3.703697e-08,8.888874e-01,...,3.703697e-08,3.703697e-08,3.703697e-08,3.703697e-08,1.111110e-01,3.703697e-08,3.703697e-08,3.703697e-08,3.703697e-08,3.703697e-08
TO,7.342818e-02,4.084442e-02,4.589261e-10,4.589261e-10,4.589261e-10,4.589261e-10,7.342818e-03,8.260670e-03,3.579624e-02,1.298761e-01,...,4.589261e-10,4.589261e-10,4.589261e-10,4.589261e-10,1.514456e-02,4.589261e-10,4.589261e-10,4.589261e-10,4.589261e-03,4.589261e-10
RB,3.438497e-02,3.190358e-03,3.792981e-02,1.417937e-03,3.544846e-04,1.524282e-02,7.054236e-02,2.197802e-02,8.153137e-03,5.175470e-02,...,3.544842e-10,1.417937e-03,3.544842e-10,6.522510e-02,1.063453e-03,3.544842e-10,3.544842e-10,4.112017e-02,1.630627e-02,3.544846e-04
-NONE-,2.730997e-03,3.762707e-02,3.868912e-02,1.517222e-04,1.517220e-10,1.838871e-01,2.366864e-02,7.495069e-02,2.882719e-03,5.340616e-02,...,1.517220e-10,1.517222e-04,1.517222e-04,3.064785e-02,4.703384e-03,1.517220e-10,1.517220e-10,9.133667e-02,1.668943e-03,1.517220e-10
$,9.889502e-01,1.381215e-09,1.381215e-09,1.381215e-09,1.381215e-09,1.381215e-09,1.381215e-09,1.381215e-09,1.381215e-09,1.381215e-09,...,1.381215e-09,1.381215e-09,1.381215e-09,1.381215e-09,1.381215e-09,1.381215e-09,1.381215e-09,1.381215e-09,1.381215e-09,1.381215e-09
DT,2.339539e-02,1.286134e-01,5.512004e-03,8.574229e-04,1.224890e-10,2.449781e-04,8.941695e-03,1.837335e-03,8.451739e-03,1.224890e-03,...,1.224891e-04,3.674670e-04,1.224890e-10,1.837335e-03,1.224891e-04,1.224890e-10,1.224890e-10,1.102401e-03,5.879471e-03,3.429691e-03


In [24]:
# use the log of probabilities here, so that the 'decode' function works for HMMs
tag_indices = decode(np.log(df_pi.values), np.log(df_trans.values),
                     np.log(df_emission[sent].values))

In [25]:
np.asarray(tags)[tag_indices]

array(['NN', 'NNS', 'IN', 'NN'], dtype='<U6')

In [26]:
sent

['eye', 'drops', 'off', 'shelf']

Explanations of the sentence 'eye drops off shelf' from http://www.vodppl.upm.edu.my/uploads/docs/Analysis%20of%20Ambiguity.pdf#page=2

Eye drops off shelf (Headlines)

Lexical ambiguity: Two possible interpretations - ‘eyedrops’ refers to the cleansing liquid
used to relieve eyes, in this case, the sentence means the items have been taken off the
market; ‘drops’ refer to the verb to fall from certain height; here it means an eye ball has
dropped off a shelf.